In [1]:
import os
import time
import tensorflow as tf
import scipy.io as sio 
import os
import scipy.io
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
from keras.layers import Input, Add, Dense, Dropout, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,concatenate,Cropping2D, MaxPool2D
from keras.models import Model
import time
import matplotlib.pyplot as plt



c:\Users\USER\anaconda3\envs\MNIST\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\Users\USER\anaconda3\envs\MNIST\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\Users\USER\anaconda3\envs\MNIST\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\Users\USER\anaconda3\envs\MNIST\lib\site-packages\tensorflow\python\frame

In [2]:

def one_hot_matrix(labels, C):
    
    ### START CODE HERE ###
    
    # Create a tf.constant equal to C (depth), name it 'C'.
    depth = tf.constant(C, name = "C")
    
    # Use tf.one_hot, be careful with the axis (approx. 1 line)
    one_hot_matrix = tf.one_hot(labels, depth, axis=0)
    
    # Create the session
    sess = tf.Session()
    
    # Run the session
    one_hot = sess.run(one_hot_matrix)
    
    # Close the session.
    sess.close() 
    
    return one_hot

In [3]:
file_path = os.getcwd() + '\\train'
label_name = os.listdir(file_path)
class_num = 3

matfn='train_easy.mat' 
data=sio.loadmat(matfn) 

X_train = data['X_data'] 
Y_train = data['Y_data']

"""
X_val = X_train[4200:]
Y_val = Y_train[4200:]

X_train = X_train[:4200]
Y_train = Y_train[:4200]
"""

print(X_train.shape)
print(Y_train.shape)

(2520, 32, 32, 3)
(1, 2520)


In [4]:
print(Y_train)

[[0 0 0 ... 2 2 2]]


In [5]:
data_num = X_train.shape[0] 
Y_train = one_hot_matrix(Y_train, class_num)
Y_train = Y_train.T
Y_train = Y_train.reshape(data_num, class_num)
print(Y_train.shape)

(2520, 3)


In [6]:
import random
num_examples = X_train.shape[0]
index_list = list(range(num_examples))
random.shuffle(index_list)

# 根据打乱后的索引重新排列训练数据和标签
X_train = X_train[index_list]
Y_train = Y_train[index_list]

In [7]:
print(Y_train)

[[0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 ...
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]


In [8]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation = 'relu', input_shape = (32, 32, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    # tf.keras.layers.Conv2D(128, (3, 3), activation = 'relu'),
    # tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation = 'relu'),
    # tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3, activation = 'softmax')
])

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 16)        448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
dropout (Dropout)            (None, 6, 6, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)    

In [10]:
import keras
from keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau

# early_stopping : val_loss 連續不降低時，結束訓練
early_stopping = EarlyStopping(monitor='val_loss', patience=15, verbose=1,restore_best_weights=True,mode='auto')
# reduce_lr : val_loss 連續不降低時，減少learning rate
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, mode='auto',
                            epsilon=0.0001, cooldown=0, min_lr=0)

checkpoint = ModelCheckpoint('small_v1.h5', monitor='val_loss', verbose=1,
    save_best_only=True, mode='auto', period=1)

# model = MobileNet(input_shape = (224, 224, 3), weights=None, classes=class_num)

###################################################################################################################################

# optimizer = keras.optimizers.Adam(lr=0.0001)
model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=["accuracy"])
history = model.fit(x=X_train, y=Y_train, validation_split=0.2, epochs=50, batch_size = 32, callbacks=[early_stopping,checkpoint,reduce_lr])

# complete_time=time.strftime("%Y_%m_%d %H_%M_%S", time.localtime()) 
# model.save('MobileNet_'+str(complete_time)+'.h5')


Train on 2016 samples, validate on 504 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/50


AttributeError: 'EarlyStopping' object has no attribute 'on_train_batch_begin'

In [ ]:
model.predict(X_train)